In [ ]:

import numpy as np
import matplotlib.pyplot as plt

import json
from tqdm.notebook import tqdm

from scipy import stats

import glob

from combra import data, angles
import os

import glob
import multiprocessing

import numpy as np
from pathlib import Path
from mpire import WorkerPool
from skimage import io, color, filters, morphology, util
import cv2
import re
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm

from combra import data, angles, mvee

In [ ]:
types_dict = {
    "Ultra_Co11": "средние зерна",
    "Ultra_Co25": "мелкие зерна",
    "Ultra_Co8": "средне-мелкие зерна",
    "Ultra_Co6_2": "крупные зерна",
    "Ultra_Co15": "средне-мелкие зерна",
}

# path = '/home/david/mnt/ssd_2_sata/python/phd/datasets/original/o_bc_left_3'
# path = './data/san_256x256_N100_000.h5'
# path = './data/san_512x512_N100_000.h5'
path = './data/o_bc_left_3.h5'

# save_path = 'san_512x512_N100_000_msl0'
save_path = 'o_bc_left_3_msl0'


dataset = data.PobeditDataset(path=path, max_images_num_per_class=100)

out = dataset.generate_angles(
    save_path=save_path,   
    types_dict=types_dict,
    step=[0.1, 0.5, 1,2,3,4,5],                     
    workers=20,
    angles_tol=3,
    min_segment_len=0.0               
)

print("Angles outputs:", out)


In [ ]:
path = "./o_bc_left_3_msl0/angles_n90.parquet"
# path = "./o_bc_left_3_msl5/angles_n90.parquet"
# path = "./o_bc_left_3_msl10/angles_n90.parquet"
# path = "./san_512x512_N100_000_msl0/angles_n100.parquet"

angles.angles_plot_base(parquet_path=path, N=10, M=7, font_size=20, scatter_size=5,  step=2, save=False,
                        ylim=[0,0.03])

In [ ]:
# %%time
# ---- read parquet and plot ----
# in_path = "./san_512x512_N100_000/angles_n10.parquet"
# in_path = "./san_256x256_N100_000/angles_n100000.parquet"

paths = ["./san_512x512_N100_000/angles_n10.parquet",
         "./san_512x512_N100_000/angles_n100.parquet",
         "./san_512x512_N100_000/angles_n1000.parquet",
         "./san_512x512_N100_000/angles_n10000.parquet",
         "./san_512x512_N100_000/angles_n100000.parquet",
         "./san_256x256_N100_000/angles_n10.parquet",
         "./san_256x256_N100_000/angles_n100.parquet",
         "./san_256x256_N100_000/angles_n1000.parquet",
         "./san_256x256_N100_000/angles_n10000.parquet",
         "./san_256x256_N100_000/angles_n100000.parquet",
         ]

# names_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}

# step 1
# 10      - [0,0.011]
# 100     - [0,0.011]
# 1_000   - [0,0.011]
# 10_000  - [0,0.011]
# 100_000 - [0,0.011]

for path in paths:

    angles.angles_plot_base(parquet_path=path, N=10, M=7, font_size=20, scatter_size=5,  step=5, save=True, ylim=[0,0.05])


In [ ]:
angles.angles_plot_base(rows, save_name="biba", N=10, M=7, save=False, indices=None, font_size=20, scatter_size=20)

In [ ]:
mvee.plot_beam_base(rows,save_name='biba',step=5,N=7,M=7,  save=False,indices=None, font_size=20,scatter_size=20)

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
from pathlib import Path

# Class name mapping
NAME_MAP = {
    "class_0": "Ultra_Co25",
    "class_1": "Ultra_Co11",
    "class_2": "Ultra_Co6_2",
}

def update_meta_names(pq_path):
    """Update meta.name in parquet file, reading only meta column for the mapping."""
    # Read only meta column to get names
    meta_table = pq.read_table(pq_path, columns=["meta"])
    meta_list = meta_table["meta"].to_pylist()
    
    # Check if any names need updating
    needs_update = any(m["name"] in NAME_MAP for m in meta_list)
    if not needs_update:
        return False
    
    # Now read full file and update
    table = pq.read_table(pq_path)
    
    # Update names in meta
    new_meta = []
    for m in meta_list:
        m_copy = dict(m)
        if m_copy["name"] in NAME_MAP:
            m_copy["name"] = NAME_MAP[m_copy["name"]]
        new_meta.append(m_copy)
    
    # Replace meta column
    meta_array = pa.array(new_meta, type=table.schema.field("meta").type)
    col_idx = table.schema.get_field_index("meta")
    table = table.set_column(col_idx, "meta", meta_array)
    
    # Write back
    pq.write_table(table, pq_path)
    return True

# Folders to process
folders = [
    Path("./san_512x512_N100_000"),
    Path("./san_256x256_N100_000"),
]

for folder in folders:
    parquets = sorted(folder.glob("*.parquet"))
    print(f"\nProcessing {len(parquets)} files in {folder}")
    
    for pq_path in parquets:
        updated = update_meta_names(pq_path)
        status = "Updated" if updated else "Skipped (no matching names)"
        print(f"  {status}: {pq_path.name}")

print("\nDone!")
